In [1]:
import tensorflow as tf
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
import keras_tuner as kt

Using TensorFlow backend


In [2]:
# load train, test, validation splits
saved_dir = 'D:/JATHURSH/Jupyter_notebook_projects/DrowsinessDetectionOnCombinedDataset/saved_data/train_test_val_splits_preprocessed_combined_dataset/'

X_train = np.load(saved_dir + 'X_train.npy')

X_val = np.load(saved_dir + 'X_val.npy')

y_train = np.load(saved_dir + 'y_train.npy')

y_val = np.load(saved_dir + 'y_val.npy')

In [3]:
def build_model(hp):
    # pretrained model - MobileNetV2
    # all should be numpy arrays or tensors before giving it to MobileNetV2
    """
    tf.keras.applications.mobilenet_v2.MobileNetV2(
    input_shape=None,                 # Shape of input data (MobileNets support any input size greater than 32 x 32, with larger image sizes offering better performance.) 
    alpha=1.0,                        # Width multiplier for controlling network width (default is 1.0)
    include_top=True,                 # Include fully connected top layers (default is True)
    weights='imagenet',               # Weight initialization ('imagenet' for pre-trained weights, None for random initialization)
    input_tensor=None,                # Optional input tensor (default is None)
    pooling=None,                     # Pooling type for the last layer ('avg', 'max', or None; default is None)
    classes=1000,                     # Number of output classes (default is 1000 for ImageNet)
    classifier_activation='softmax',   # Activation function for output layer (default is 'softmax')
    **kwargs                          # Additional keyword arguments
    )
    """
    
    # we dont have to set anything and keep its default values
    MobileNetV2_model = tf.keras.applications.mobilenet_v2.MobileNetV2()
    
    # Make all layers in the model non-trainable - freeze
    for layer in MobileNetV2_model.layers:
        layer.trainable = False
        
    base_input = MobileNetV2_model.layers[0].input
    base_output = MobileNetV2_model.layers[-3].output
    global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_output)
    
    # Hyperparameter: Include Dense Layer (True or False)
    include_dense = hp.Boolean('include_dense')

    if include_dense:
        # Hyperparameter: Number of neurons in the dense layer
        num_neurons = hp.Int('num_neurons', min_value=32, max_value=512, step=32)
        dense_layer = tf.keras.layers.Dense(num_neurons, activation='relu')(global_average_layer)
        # Hyperparameter: Dropout rate
        '''
        Dropout is a regularization technique that helps prevent overfitting by randomly 
        setting a fraction of the input units to 0 during training.
        '''
        dropout_rate = hp.Float('dropout_rate', min_value=0.0, max_value=0.5, step=0.1)
        dropout_layer = tf.keras.layers.Dropout(dropout_rate)(dense_layer)
        final_output = tf.keras.layers.Dense(1)(dropout_layer)
    else:
        final_output = tf.keras.layers.Dense(1)(global_average_layer)
        
    final_output = tf.keras.layers.Activation("sigmoid")(final_output)
    
    drowsiness_detection_model = tf.keras.Model(inputs=base_input, outputs=final_output)
    
    # Hyperparameter: Learning rate
    learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    drowsiness_detection_model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    return drowsiness_detection_model

In [4]:
# Initialize the Keras Tuner RandomSearch tuner
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',  # The metric to optimize
    max_trials=10,             # Number of hyperparameter combinations to try
    directory='tuner_dir',      # Directory for saving results
    project_name='drowsiness_detection_tuner'  # Name of the project
)

In [5]:
# Hyperparameter: Batch size
batch_size = 32 # hp.Int('batch_size', min_value=16, max_value=128, step=16)

# Use the ReduceLROnPlateau learning rate scheduler
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-6,
        verbose=1
)
    
# Start the hyperparameter search using your training data with early stopping
tuner.search(X_train, y_train,
             epochs=100, 
             validation_data=(X_val, y_val), 
             batch_size=batch_size,
             callbacks=[tf.keras.callbacks.EarlyStopping(patience=5), lr_scheduler])

Trial 10 Complete [03h 04m 41s]
val_accuracy: 0.9693187475204468

Best val_accuracy So Far: 0.9694488048553467
Total elapsed time: 2d 15h 10m 58s


In [6]:
tuner.results_summary()

Results summary
Results in tuner_dir\drowsiness_detection_tuner
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 03 summary
Hyperparameters:
include_dense: True
learning_rate: 0.001
num_neurons: 224
dropout_rate: 0.0
Score: 0.9694488048553467

Trial 09 summary
Hyperparameters:
include_dense: True
learning_rate: 0.001
num_neurons: 416
dropout_rate: 0.4
Score: 0.9693187475204468

Trial 08 summary
Hyperparameters:
include_dense: True
learning_rate: 0.0001
num_neurons: 288
dropout_rate: 0.2
Score: 0.9680187106132507

Trial 05 summary
Hyperparameters:
include_dense: True
learning_rate: 0.0001
num_neurons: 192
dropout_rate: 0.0
Score: 0.9677587151527405

Trial 02 summary
Hyperparameters:
include_dense: True
learning_rate: 0.001
num_neurons: 32
dropout_rate: 0.0
Score: 0.9673687219619751

Trial 00 summary
Hyperparameters:
include_dense: False
learning_rate: 0.0001
Score: 0.9619084596633911

Trial 06 summary
Hyperparameters:
include_dense: False
learning_rate: 0.00

When include_dense is set to False, the values for num_neurons and dropout_rate are still generated as hyperparameters, but they won't be used in creating the dense layer or dropout layer in the model. In this case, those hyperparameters are effectively placeholders, and their values are not used because you've excluded the dense layer from the model.

In [12]:
tuner.get_best_hyperparameters()[0].values

{'include_dense': True,
 'learning_rate': 0.001,
 'num_neurons': 224,
 'dropout_rate': 0.0}